# Data Preprocessing

In [ ]:
import pandas as pd 
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEndoer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif